In [ ]:
import os
import json
import collections
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

from image_captioning.constants import DATA_DIR
from image_captioning.data_pipeline import input_dataset

# Download data

This can take some time as it's ~13GB of data.

In [ ]:
# Download caption annotation files
if not os.path.exists(DATA_DIR):
    annotation_zip = tf.keras.utils.get_file(
        fname='captions.zip',
        origin='http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
        cache_subdir=DATA_DIR,
        extract=True
    )
    os.remove(annotation_zip)

annotation_file = os.path.join(DATA_DIR, 'annotations/captions_train2014.json')

In [ ]:
# Download image files
image_folder = os.path.join(DATA_DIR, 'train2014')
if not os.path.exists(DATA_DIR):
    image_zip = tf.keras.utils.get_file(
        fname='train2014.zip',
        origin='http://images.cocodataset.org/zips/train2014.zip',
        cache_subdir=image_folder,
        extract=True
    )
    os.remove(image_zip)

# Load data

In [ ]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

In [ ]:
# Group all captions together having the same image ID.
image_path_to_caption = collections.defaultdict(list)
for val in annotations['annotations']:
    caption = f"<start> {val['caption']} <end>"
    image_path = os.path.join(image_folder, 'COCO_train2014_{:012d}.jpg'.format(val['image_id']))
    image_path_to_caption[image_path].append(caption)

image_paths = list(image_path_to_caption.keys())

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

random_image_path = np.random.choice(image_paths)
ax.set_title("\n".join(image_path_to_caption[random_image_path]), fontsize=14)
ax.imshow(plt.imread(random_image_path));

# Data preprocess image

In [ ]:
random_image_path = np.random.choice(image_paths)
preprocessed_img, _ = input_dataset.load_and_preprocess_image(random_image_path)

print(preprocessed_img.shape)
plt.imshow((preprocessed_img + 1) / 2);

# Encode images

In [ ]:
train_captions = []
img_name_vector = []

for image_path in image_paths:
    caption_list = image_path_to_caption[image_path]
    train_captions.extend(caption_list)
    img_name_vector.extend([image_path] * len(caption_list))  # duplicate image path so that every caption has its own image path

In [ ]:
inceptionV3 = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [ ]:
# get unique image paths
encode_train = sorted(set(img_name_vector))

# create dataset that returns images and their corresponding filepaths on disk
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(input_dataset.load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

In [ ]:
os.makedirs(os.path.join(DATA_DIR, 'train2014_features'), exist_ok=True)

for batch_imgs, batch_paths in tqdm(image_dataset):
    
    batch_features = inceptionV3(batch_imgs)  # output shape (bs, 8, 8, 2048)
    batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))  # output shape (bs, 64, 2048); basically flattens the spatial dimension

    for bf, p in zip(batch_features, batch_paths):
        img_path = p.numpy().decode("utf-8")
        filename = img_path.split('/')[-1].split('.')[0]
        output_path = os.path.join(DATA_DIR, 'train2014_features', filename)  # store features in separate directory
        np.save(output_path, bf.numpy())  # will append .npy to output filename